### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v1.0.60' # also set this in conda file below

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/workspace/amlsdk/docker_build'

'/workspace/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60'

'/workspace/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain]==1.0.60 

Writing /workspace/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.7.10 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gpg-agent \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.1.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /workspace/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/workspace/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60'

'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60'

'georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.60'

'georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.60'

'/workspace/amlsdk/docker_build'

total 5
-rwxr-xr-x 1 root root  322 Sep 10 21:55 aml_sdk_conda_dep_file.yml
-rwxr-xr-x 1 root root 1841 Sep 10 21:55 dockerfile_aml-sdk_docker_image_sdk.v1.0.60


In [9]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
build_command

!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path


'docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60 --target AzureMLSDKOnly -f /workspace/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.60 /workspace/amlsdk/docker_build --no-cache'

/bin/sh: 1: docker: Permission denied
/bin/sh: 1: docker: Permission denied
/bin/sh: 1: docker: Permission denied


In [11]:
import docker

cli = docker.APIClient()
response=''
response = [line for line in cli.build(
    path=working_path, 
    dockerfile=docker_file_path, 
    pull=True,
    rm=True, 
    quiet=False,
    nocache=True,
    tag=aml_docker_image_name
)]

In [12]:
response[100:]

[b'{"stream":"(Reading database ... 45%\\r"}\r\n',
 b'{"stream":"(Reading database ... 50%\\r"}\r\n',
 b'{"stream":"(Reading database ... 55%\\r"}\r\n',
 b'{"stream":"(Reading database ... 60%\\r"}\r\n',
 b'{"stream":"(Reading database ... 65%\\r"}\r\n',
 b'{"stream":"(Reading database ... 70%\\r"}\r\n',
 b'{"stream":"(Reading database ... 75%\\r"}\r\n',
 b'{"stream":"(Reading database ... 80%\\r"}\r\n',
 b'{"stream":"(Reading database ... 85%\\r"}\r\n',
 b'{"stream":"(Reading database ... 90%\\r"}\r\n',
 b'{"stream":"(Reading database ... 95%\\r"}\r\n',
 b'{"stream":"(Reading database ... 100%\\r(Reading database ... 12557 files and directories currently installed.)\\r\\n"}\r\n',
 b'{"stream":"Preparing to unpack .../libpython3.7-minimal_3.7.3-2_amd64.deb ...\\r\\n"}\r\n',
 b'{"stream":"Unpacking libpython3.7-minimal:amd64 (3.7.3-2) ...\\r\\n"}\r\n',
 b'{"stream":"Selecting previously unselected package python3.7-minimal.\\r\\nPreparing to unpack .../python3.7-minimal_3.7.3-2_amd64.de

In [13]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir02']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/workspace/amlsdk/docker_run_dir02'

total 0


In [20]:
client = docker.from_env()
client.containers.list()

[<Container: 6759c847db>]

In [29]:
# export the resulted conda env

cli_command_inside_container = ' /bin/bash -c "conda env list ; pwd; ls -l /workspace ; ' + \
    'python -c \'import azureml.core;print(azureml.core.VERSION)\'; ' + \
    'conda env export > /workspace/exported_conda_env.yml"'

cli_command='docker run -it --rm  --name aml-sdk_docker_container ' + \
'-v ' + crt_local_dir + ':/workspace:rw ' + \
aml_docker_image_name + \
cli_command_inside_container

cli_command

if 0:
    ! $cli_command
else:
    client = docker.from_env()
    client.containers.run(aml_docker_image_name, 
                          runtime='',
                          remove=True,
                          volumes={'/workspace': {'bind': '/workspace', 'mode': 'rw'}},
                          working_dir='/',
                          command=cli_command_inside_container)

'docker run -it --rm  --name aml-sdk_docker_container -v /workspace/amlsdk/docker_run_dir02:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60 /bin/bash -c "conda env list ; pwd; ls -l /workspace ; python -c \'import azureml.core;print(azureml.core.VERSION)\'; conda env export > /workspace/exported_conda_env.yml"'

ContainerError: Command ' /bin/bash -c "conda env list ; pwd; ls -l /workspace ; python -c 'import azureml.core;print(azureml.core.VERSION)'; conda env export > /workspace/exported_conda_env.yml"' in image 'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.60' returned non-zero exit status 1: b'\n# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<\n\n    Traceback (most recent call last):\n      File "/opt/conda/lib/python3.7/site-packages/conda/exceptions.py", line 1062, in __call__\n        return func(*args, **kwargs)\n      File "/opt/conda/lib/python3.7/site-packages/conda_env/cli/main.py", line 73, in do_call\n        exit_code = getattr(module, func_name)(args, parser)\n      File "/opt/conda/lib/python3.7/site-packages/conda_env/cli/main_export.py", line 94, in execute\n        ignore_channels=args.ignore_channels)\n      File "/opt/conda/lib/python3.7/site-packages/conda_env/env.py", line 96, in from_environment\n        precs = tuple(PrefixGraph(pd.iter_records()).gr

In [ ]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [ ]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [ ]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


#### Do the same in a  notebook

In [ ]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [ ]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


In [ ]:
!ls -l $crt_local_dir

#### Run the notebook and see the new SDK version


In [ ]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

In [ ]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [ ]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

In [ ]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb